In [28]:
import pandas as pd
import numpy as np
import warnings

#------Читаем файл продаж за последний месяц ---------------------

with warnings.catch_warnings(record=True):
    warnings.simplefilter("always")
    df = pd.read_excel('MouthSales.XLSX', engine="openpyxl")
    
#------удаляем их header, сдвигаем начало df на один вниз----------

df.columns = df.iloc[0]
df = df[1:]
df.reset_index(drop=True)
print('----Before----')
print(df.columns)

#-----переименовываем колонки так, чтобы там не было 2 раза'шт '--

df.columns = ['Бренд', 'Предмет', 'Сезон', 'Коллекция', 'Наименование',
       'Артикул продавца', 'Артикул WB', 'Баркод', 'Размер', 'Контракт',
       'Склад', 'Поступило', 'Заказано штук', 'Сумма заказов минус комиссия WB, руб.',
       'Выкупили, шт.', 'К перечислению за товар, руб.',
       'Текущий остаток, шт.']
print('----After----')
print(df.columns)


----Before----
Index(['Бренд', 'Предмет', 'Сезон', 'Коллекция', 'Наименование',
       'Артикул продавца', 'Артикул WB', 'Баркод', 'Размер', 'Контракт',
       'Склад', 'шт.', 'шт.', 'Сумма заказов минус комиссия WB, руб.',
       'Выкупили, шт.', 'К перечислению за товар, руб.',
       'Текущий остаток, шт.'],
      dtype='object', name=0)
----After----
Index(['Бренд', 'Предмет', 'Сезон', 'Коллекция', 'Наименование',
       'Артикул продавца', 'Артикул WB', 'Баркод', 'Размер', 'Контракт',
       'Склад', 'Поступило', 'Заказано штук',
       'Сумма заказов минус комиссия WB, руб.', 'Выкупили, шт.',
       'К перечислению за товар, руб.', 'Текущий остаток, шт.'],
      dtype='object')


In [29]:
#-------Grouping by Артикул продавца -----
df_pivot = df.pivot_table(values=['Заказано штук'], index='Артикул продавца', aggfunc = 'sum').sort_values(
                                      by=['Заказано штук'], ascending=False)
#-------let's find speed of buying -------
days = 30
df_pivot['Speed'] = round((df_pivot['Заказано штук'] / 30), 2)
df_pivot

,Заказано штук,Speed
Артикул продавца,,
Черный эла,38,1.27
Золотистый прямоугольник,26,0.87
Серебряный прямоугольник,23,0.77
Серый эла,17,0.57
Синий эла,12,0.40
Эластичный ремень,11,0.37
Серебряный круглый,10,0.33


In [30]:
#-----let's find left goods minus goods that are on the way ------

with warnings.catch_warnings(record=True):
    warnings.simplefilter("always")
    df_left = pd.read_excel('left.xlsx', engine="openpyxl")
    
df_left_pivot = df_left.pivot_table(
    values=['Итого по складам', 'Товары в пути до клиента', 'Товары в пути от клиента'], 
    index='Артикул продавца', 
    aggfunc = 'sum')

df_left_pivot['Left'] = df_left_pivot['Итого по складам'] - df_left_pivot['Товары в пути до клиента'] - df_left_pivot['Товары в пути от клиента']
df_left_pivot

,Итого по складам,Товары в пути до клиента,Товары в пути от клиента,Left
Артикул продавца,,,,
3285,0.0,0.0,1.0,-1.0
Золотистый прямоугольник,20.0,3.0,0.0,17.0
Серебряный круглый,32.0,3.0,0.0,29.0
Серебряный прямоугольник,10.0,3.0,1.0,6.0
Серый эла,9.0,3.0,0.0,6.0
Синий эла,12.0,1.0,0.0,11.0
Черный эла,23.0,1.0,0.0,22.0
Эластичный ремень,20.0,1.0,0.0,19.0


In [31]:
#----------------Combining two tables -----------------
result = pd.merge(df_pivot, df_left_pivot, on="Артикул продавца")
result.drop(columns = ['Заказано штук', 'Итого по складам', 'Товары в пути до клиента', 'Товары в пути от клиента'], axis = 1, 
            inplace = True) 
result

,Speed,Left
Артикул продавца,,
Черный эла,1.27,22.0
Золотистый прямоугольник,0.87,17.0
Серебряный прямоугольник,0.77,6.0
Серый эла,0.57,6.0
Синий эла,0.40,11.0
Эластичный ремень,0.37,19.0
Серебряный круглый,0.33,29.0


In [32]:
#------------- How much to deliver so it is okey for next 60 days  --------
result['To deliver'] = result['Speed'] * 60 + 10 - result['Left']

#------------- Find a week product ----------------------------------------
result['Weak product'] = np.where(result['Speed'] > 0.5, False, True)

result

,Speed,Left,To deliver,Weak product
Артикул продавца,,,,
Черный эла,1.27,22.0,64.2,False
Золотистый прямоугольник,0.87,17.0,45.2,False
Серебряный прямоугольник,0.77,6.0,50.2,False
Серый эла,0.57,6.0,38.2,False
Синий эла,0.40,11.0,23.0,True
Эластичный ремень,0.37,19.0,13.2,True
Серебряный круглый,0.33,29.0,0.8,True


In [33]:
result.to_excel("To deliver.xlsx") 